# Sequence Optimization - Scenario 3

- GA
- Move limit v02 - With GA Looping & max_failures = 10


In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import random

from ortools.sat.python import cp_model


C:\Users\msubki\AppData\Local\Temp\ipykernel_23160\2469439399.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Import Data


In [137]:
root = "/Users/finazaria/Documents/03 Deloitte/03e Data"

In [2]:
alc = pd.read_csv(f"alc-prd_stg.stg_datacenter_t_datacenter-aug24_2.csv", delimiter=";")
alc.head()

,ref_no,body_no,id_no,vin_no,curr_tp,bodyseq_w,bodyseq_a,model_no,p_sfx,s_sfx,...,asj0,ask0,asn0,asq0,asv1,asg1,asv2,asc1,prc_ts,source_system
0,AD48019531,56193,DH08160078,MHFAB1BY0S3115520,4Q0,902,825.0,W101LE-LBGF,2J,A1,...,2.0,2.0,1.0,1.0,NaN,2.0,NaN,NaN,"2,02408E+13",ALC
1,AD48019533,53510,DH08090077,MHFAB1BY6S3114551,4Q0,219,293.0,W101LE-LBGF,2J,A1,...,2.0,2.0,1.0,1.0,NaN,2.0,NaN,NaN,"2,02408E+13",ALC
2,AD48019535,50603,DH08010068,MHFAB1BY6S3113514,4Q0,312,233.0,W101LE-LBGF,2J,A1,...,2.0,2.0,1.0,1.0,NaN,2.0,NaN,NaN,"2,02408E+13",ALC
3,AD48019538,53060,DH08080077,MHFAB1BY8S3114390,4Q0,769,717.0,W101LE-LBGF,2J,A1,...,2.0,2.0,1.0,1.0,NaN,2.0,NaN,NaN,"2,02408E+13",ALC
4,AD48019539,54630,DH08130070,MHFAB1BY5S3114959,4Q0,339,293.0,W101LE-LBGF,2J,A1,...,2.0,2.0,1.0,1.0,NaN,2.0,NaN,NaN,"2,02408E+13",ALC


In [3]:
def check_null(df):
    total_entries = len(df)  # Total number of entries in the DataFrame

    # Calculate null counts for each column
    null_data = df.isnull().sum()

    # Calculate percentage of nulls for each column
    percent_null = (null_data / total_entries * 100).round(2)

    # Create a DataFrame to display null counts and percentages
    result = pd.DataFrame({
        'Null Counts': null_data,
        'Percentage Null': percent_null.astype(str) + '%'
    })
    
    return result

In [ ]:
pd.set_option('display.max_rows', None)
check_null(alc)

In [5]:
pd.reset_option('display.max_rows')

## Data Preperation


In [6]:
total_prod = alc['vin_no'].nunique()
total_prod

16873

In [7]:
alc['plant_no'] = np.where(alc['curr_tp'].str.startswith('1'), 1, 2)
alc.head()

,ref_no,body_no,id_no,vin_no,curr_tp,bodyseq_w,bodyseq_a,model_no,p_sfx,s_sfx,...,ask0,asn0,asq0,asv1,asg1,asv2,asc1,prc_ts,source_system,plant_no
0,AD48019531,56193,DH08160078,MHFAB1BY0S3115520,4Q0,902,825.0,W101LE-LBGF,2J,A1,...,2.0,1.0,1.0,NaN,2.0,NaN,NaN,"2,02408E+13",ALC,2
1,AD48019533,53510,DH08090077,MHFAB1BY6S3114551,4Q0,219,293.0,W101LE-LBGF,2J,A1,...,2.0,1.0,1.0,NaN,2.0,NaN,NaN,"2,02408E+13",ALC,2
2,AD48019535,50603,DH08010068,MHFAB1BY6S3113514,4Q0,312,233.0,W101LE-LBGF,2J,A1,...,2.0,1.0,1.0,NaN,2.0,NaN,NaN,"2,02408E+13",ALC,2
3,AD48019538,53060,DH08080077,MHFAB1BY8S3114390,4Q0,769,717.0,W101LE-LBGF,2J,A1,...,2.0,1.0,1.0,NaN,2.0,NaN,NaN,"2,02408E+13",ALC,2
4,AD48019539,54630,DH08130070,MHFAB1BY5S3114959,4Q0,339,293.0,W101LE-LBGF,2J,A1,...,2.0,1.0,1.0,NaN,2.0,NaN,NaN,"2,02408E+13",ALC,2


In [8]:
# Get sequence of cars in G0 (One station before J0)
g0 = alc[(alc['curr_tp'] == '1G0') | (alc['curr_tp'] == '4G0')]
prod_g0 = len(g0)
print('Number of vehicle in J0:', prod_g0)


Number of vehicle in J0: 198


In [9]:
# Take only G0 from plant 1
g0_1 = g0[g0['plant_no'] == 1]
prod_g0_1 = len(g0_1)
print('Number of vehicle in G0 - Plant 1:', prod_g0_1)

Number of vehicle in G0 - Plant 1: 94


In [10]:
g0_1.head()

,ref_no,body_no,id_no,vin_no,curr_tp,bodyseq_w,bodyseq_a,model_no,p_sfx,s_sfx,...,ask0,asn0,asq0,asv1,asg1,asv2,asc1,prc_ts,source_system,plant_no
803,AD48023087,22309,D908210147,MHFKX8FS8R0321930,1G0,250,NaN,TGN156L-SDTSKV,DB,B1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,02408E+13",ALC,1
4018,AD48027696,22376,D108210095,MHFJB8EM0R1141865,1G0,317,NaN,GUN142R-MDTSXD,JG,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,02408E+13",ALC,1
7987,AD48036684,22380,D908210034,MHFAA3GS6R0751244,1G0,321,NaN,GUN166L-SDTHXM,CK,F4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,02408E+13",ALC,1
9050,AD48036474,22353,D908210014,MHFBA3FS5R1055961,1G0,294,NaN,GUN156L-SDTHXM,BN,F7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,02408E+13",ALC,1
12539,AD48022855,22556,D908220052,MHFBU3FS4S0352903,1G0,497,NaN,GGN155L-SDTHK,AA,P7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,02408E+13",ALC,1


In [11]:
# Take only important columns

g0_column = ['body_no', 'vin_no', 'curr_tp', 'model_no', 'p_sfx', 's_sfx', 'ext_color', 'proc_date_g0', 'proc_time_g0', 'plan_date_j0', 'plan_time_j0',
                'plan_date_k0', 'plan_time_k0'
]

In [12]:
# Take data from plant 1 for example

g0_1_cleaned = g0_1[g0_column]
g0_1_cleaned.head()

,body_no,vin_no,curr_tp,model_no,p_sfx,s_sfx,ext_color,proc_date_g0,proc_time_g0,plan_date_j0,plan_time_j0,plan_date_k0,plan_time_k0
803,22309,MHFKX8FS8R0321930,1G0,TGN156L-SDTSKV,DB,B1,89,20240820.0,30743.0,20240820,45500,20240820,52800
4018,22376,MHFJB8EM0R1141865,1G0,GUN142R-MDTSXD,JG,20,218,20240820.0,53116.0,20240821,85100,20240821,92400
7987,22380,MHFAA3GS6R0751244,1G0,GUN166L-SDTHXM,CK,F4,1D6,20240820.0,53820.0,20240821,85900,20240821,94200
9050,22353,MHFBA3FS5R1055961,1G0,GUN156L-SDTHXM,BN,F7,2PS,20240821.0,103557.0,20240821,124100,20240821,131400
12539,22556,MHFBU3FS4S0352903,1G0,GGN155L-SDTHK,AA,P7,218,20240821.0,110505.0,20240821,150700,20240821,155200


In [13]:
# Check for date
g0_1_cleaned['proc_date_g0'].unique()

array([20240820., 20240821.])

In [14]:
# Take the date 20240821
g0_1_cleaned = g0_1_cleaned[g0_1_cleaned['proc_date_g0'] == 20240821]

In [15]:
pd.set_option('display.max_columns', None)

g0_1_cleaned.head(10)

,body_no,vin_no,curr_tp,model_no,p_sfx,s_sfx,ext_color,proc_date_g0,proc_time_g0,plan_date_j0,plan_time_j0,plan_date_k0,plan_time_k0
9050,22353,MHFBA3FS5R1055961,1G0,GUN156L-SDTHXM,BN,F7,2PS,20240821.0,103557.0,20240821,124100,20240821,131400
12539,22556,MHFBU3FS4S0352903,1G0,GGN155L-SDTHK,AA,P7,218,20240821.0,110505.0,20240821,150700,20240821,155200
12543,22517,MHFCX8GS1R0265672,1G0,TGN166L-SDTLKV,DJ,LE,1G3,20240821.0,100609.0,20240821,135800,20240821,145200
12548,22497,MHFDX8FS8R0141632,1G0,TGN156L-SDTLKV,C5,JA,89,20240821.0,92510.0,20240821,132400,20240821,135700
12554,22501,MHFDX8FSXR0141633,1G0,TGN156L-SDTLKV,C7,G4,89,20240821.0,94233.0,20240821,133200,20240821,140500
12561,22428,MHFDB8FS2R0512248,1G0,GUN155L-SDTLXV,DT,D4,89,20240821.0,80758.0,20240821,104000,20240821,111300
12565,22551,MHFABBAA5R0430464,1G0,MAGH10L-BRXHBV,DQ,L2,89,20240821.0,105545.0,20240821,150100,20240821,154500
12567,22459,MHFABBAA4R0430441,1G0,MAGH10L-BRXLBV,DT,L1,4Q0,20240821.0,82528.0,20240821,114000,20240821,125800
12570,22541,MHFABAAA4R0026341,1G0,MAGH10L-BRXLBV,CL,AH,1D6,20240821.0,104204.0,20240821,144600,20240821,153100
12580,22477,MHFJB8EMXR1141887,1G0,GUN142R-MDMSXD,HR,20,40,20240821.0,85459.0,20240821,125300,20240821,132600


In [16]:
# Sort g0 based on proc_time_g0 to know the sequence

g0_1_cleaned = g0_1_cleaned.sort_values(by='proc_time_g0')
g0_1_cleaned.head(10)

,body_no,vin_no,curr_tp,model_no,p_sfx,s_sfx,ext_color,proc_date_g0,proc_time_g0,plan_date_j0,plan_time_j0,plan_date_k0,plan_time_k0
14261,22444,MHFBA3FS9R1055963,1G0,GUN156L-SDTHXM,BN,F7,2PS,20240821.0,75846.0,20240821,153600,20240821,211400
14337,22447,MHFABAAA4R0026324,1G0,MAGH10R-BRXLBD,C4,0,218,20240821.0,80135.0,20240821,110800,20240821,114100
12561,22428,MHFDB8FS2R0512248,1G0,GUN155L-SDTLXV,DT,D4,89,20240821.0,80758.0,20240821,104000,20240821,111300
14141,22458,MHFJB8EM2R1141883,1G0,GUN142R-MDTSXD,JG,20,218,20240821.0,82406.0,20240821,113900,20240821,125700
12567,22459,MHFABBAA4R0430441,1G0,MAGH10L-BRXLBV,DT,L1,4Q0,20240821.0,82528.0,20240821,114000,20240821,125800
16635,22460,MHFDX8FS6R0141631,1G0,TGN156L-SDTLKV,C7,G4,89,20240821.0,82818.0,20240821,114100,20240821,125900
12745,22474,MHFABBAA5R0430447,1G0,MAGH10L-BPXHBU,BT,MH,89,20240821.0,85035.0,20240821,124800,20240821,132100
12600,22475,MHFJB8EM8R1141886,1G0,GUN142R-MDTSXD,JG,20,218,20240821.0,85203.0,20240821,125100,20240821,132400
12762,22476,MHFAAAAA3R0034675,1G0,MAGA10R-BRXMBD,BK,0,218,20240821.0,85327.0,20240821,125100,20240821,132400
12580,22477,MHFJB8EMXR1141887,1G0,GUN142R-MDMSXD,HR,20,40,20240821.0,85459.0,20240821,125300,20240821,132600


### Join j0 with Restriction Table


In [17]:
rest = pd.read_excel(f"Master Restriksi.xlsx")
rest.head()

,MdlXSFX,IDLINECD,Katashiki,SFX,Trim,Des,DesCode,Transmisi,Hybrid,Panoramic,Engine,GGN,CSA
0,DDC9,DD,NSP151R-CHMVKD,C9,FE26,CBU Domestic,DOM,Manual,Conventional,Non Pano,2NR-FE,Non GGN,Non CSA
1,DDYA,DD,NSP151R-CHMVKD,YA,FE26,CBU Domestic,DOM,Manual,Conventional,Non Pano,2NR-FE,Non GGN,Non CSA
2,DDC3,DD,NSP151R-CHXVKD,C3,FE26,CBU Domestic,DOM,Automatic,Conventional,Non Pano,2NR-FE,Non GGN,Non CSA
3,DDYB,DD,NSP151R-CHXVKD,YB,FE26,CBU Domestic,DOM,Automatic,Conventional,Non Pano,2NR-FE,Non GGN,Non CSA
4,DDC4,DD,NSP151R-CHXVKD,C4,FE26,CBU Domestic,DOM,Automatic,Conventional,Non Pano,2NR-FE,Non GGN,CSA


In [ ]:
# Create a column to later join j0_1_cleaned with rest table
g0_1_cleaned['katashiki_sfx_alc'] = (g0_1_cleaned['model_no'].str.strip()).str.cat(g0_1_cleaned['p_sfx'], sep="-")
g0_1_cleaned


In [19]:
# Create a new column for rest table
rest['katashiki_sfx_rest'] = (rest['Katashiki'].str.strip()).str.cat(rest['SFX'], sep="-")
rest.head()

,MdlXSFX,IDLINECD,Katashiki,SFX,Trim,Des,DesCode,Transmisi,Hybrid,Panoramic,Engine,GGN,CSA,katashiki_sfx_rest
0,DDC9,DD,NSP151R-CHMVKD,C9,FE26,CBU Domestic,DOM,Manual,Conventional,Non Pano,2NR-FE,Non GGN,Non CSA,NSP151R-CHMVKD-C9
1,DDYA,DD,NSP151R-CHMVKD,YA,FE26,CBU Domestic,DOM,Manual,Conventional,Non Pano,2NR-FE,Non GGN,Non CSA,NSP151R-CHMVKD-YA
2,DDC3,DD,NSP151R-CHXVKD,C3,FE26,CBU Domestic,DOM,Automatic,Conventional,Non Pano,2NR-FE,Non GGN,Non CSA,NSP151R-CHXVKD-C3
3,DDYB,DD,NSP151R-CHXVKD,YB,FE26,CBU Domestic,DOM,Automatic,Conventional,Non Pano,2NR-FE,Non GGN,Non CSA,NSP151R-CHXVKD-YB
4,DDC4,DD,NSP151R-CHXVKD,C4,FE26,CBU Domestic,DOM,Automatic,Conventional,Non Pano,2NR-FE,Non GGN,CSA,NSP151R-CHXVKD-C4


In [20]:
# Join two tables
g0_1_mapped = pd.merge(g0_1_cleaned, rest, 
                    left_on='katashiki_sfx_alc', 
                    right_on='katashiki_sfx_rest', 
                    how='left')

g0_1_mapped = g0_1_mapped.drop_duplicates(subset=['vin_no'])
g0_1_mapped

,body_no,vin_no,curr_tp,model_no,p_sfx,s_sfx,ext_color,proc_date_g0,proc_time_g0,plan_date_j0,plan_time_j0,plan_date_k0,plan_time_k0,katashiki_sfx_alc,MdlXSFX,IDLINECD,Katashiki,SFX,Trim,Des,DesCode,Transmisi,Hybrid,Panoramic,Engine,GGN,CSA,katashiki_sfx_rest
0,22444,MHFBA3FS9R1055963,1G0,GUN156L-SDTHXM,BN,F7,2PS,20240821.0,75846.0,20240821,153600,20240821,211400,GUN156L-SDTHXM-BN,D9BN,D9,GUN156L-SDTHXM,BN,LJ24,CBU PHILIPPINE,EXP,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,CSA,GUN156L-SDTHXM-BN
1,22447,MHFABAAA4R0026324,1G0,MAGH10R-BRXLBD,C4,0,218,20240821.0,80135.0,20240821,110800,20240821,114100,MAGH10R-BRXLBD-C4,D8C4,D8,MAGH10R-BRXLBD,C4,FB20,Domestic - IND OPTION PKG,DOM,Automatic,Hybrid,Non Pano,M20A,Non GGN,Non CSA,MAGH10R-BRXLBD-C4
3,22428,MHFDB8FS2R0512248,1G0,GUN155L-SDTLXV,DT,D4,89,20240821.0,80758.0,20240821,104000,20240821,111300,GUN155L-SDTLXV-DT,D9DT,D9,GUN155L-SDTLXV,DT,FM42,CBU SAUDI ARABIA,EXP,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,Non CSA,GUN155L-SDTLXV-DT
4,22458,MHFJB8EM2R1141883,1G0,GUN142R-MDTSXD,JG,20,218,20240821.0,82406.0,20240821,113900,20240821,125700,GUN142R-MDTSXD-JG,D1JG,D1,GUN142R-MDTSXD,JG,FB41,CBU DOMESTIC,DOM,Automatic,Conventional,Non Pano,GD-Diesel,Non GGN,Non CSA,GUN142R-MDTSXD-JG
5,22459,MHFABBAA4R0430441,1G0,MAGH10L-BRXLBV,DT,L1,4Q0,20240821.0,82528.0,20240821,114000,20240821,125800,MAGH10L-BRXLBV-DT,D8DT,D8,MAGH10L-BRXLBV,DT,FB20,UAE,EXP,Automatic,Hybrid,Non Pano,M20A,Non GGN,CSA,MAGH10L-BRXLBV-DT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,22489,MHFAA3GS1R0751247,1G0,GUN166L-SDTHXM,CL,F5,2PS,20240821.0,110015.0,20240821,221500,20240821,224800,GUN166L-SDTHXM-CL,D9CL,D9,GUN166L-SDTHXM,CL,LF31,CBU PHILIPPINE,EXP,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,CSA,GUN166L-SDTHXM-CL
127,22554,MHFKX8FS800321932,1G0,TGN156L-SDTSKN,EQ,P3,218,20240821.0,110201.0,20240821,150500,20240821,154900,TGN156L-SDTSKN-EQ,D9EQ,D9,TGN156L-SDTSKN,EQ,FN22,GENERAL,EXP,Automatic,Conventional,Non Pano,TR-Gasoline,Non GGN,Non CSA,TGN156L-SDTSKN-EQ
128,22555,MHFABAAAXR0026344,1G0,MAGH10R-BRXLBD,C4,1,89,20240821.0,110342.0,20240821,150600,20240821,155100,MAGH10R-BRXLBD-C4,D8C4,D8,MAGH10R-BRXLBD,C4,FB20,Domestic - IND OPTION PKG,DOM,Automatic,Hybrid,Non Pano,M20A,Non GGN,Non CSA,MAGH10R-BRXLBD-C4
130,22556,MHFBU3FS4S0352903,1G0,GGN155L-SDTHK,AA,P7,218,20240821.0,110505.0,20240821,150700,20240821,155200,GGN155L-SDTHK-AA,D9AA,D9,GGN155L-SDTHK,AA,FN22,CBU ECUADOR,EXP,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,CSA,GGN155L-SDTHK-AA


In [21]:
# Dropped unused columns
g0_1_mapped = g0_1_mapped.drop(['Des', 'DesCode'], axis=1)
g0_1_mapped.head(10)

,body_no,vin_no,curr_tp,model_no,p_sfx,s_sfx,ext_color,proc_date_g0,proc_time_g0,plan_date_j0,plan_time_j0,plan_date_k0,plan_time_k0,katashiki_sfx_alc,MdlXSFX,IDLINECD,Katashiki,SFX,Trim,Transmisi,Hybrid,Panoramic,Engine,GGN,CSA,katashiki_sfx_rest
0,22444,MHFBA3FS9R1055963,1G0,GUN156L-SDTHXM,BN,F7,2PS,20240821.0,75846.0,20240821,153600,20240821,211400,GUN156L-SDTHXM-BN,D9BN,D9,GUN156L-SDTHXM,BN,LJ24,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,CSA,GUN156L-SDTHXM-BN
1,22447,MHFABAAA4R0026324,1G0,MAGH10R-BRXLBD,C4,0,218,20240821.0,80135.0,20240821,110800,20240821,114100,MAGH10R-BRXLBD-C4,D8C4,D8,MAGH10R-BRXLBD,C4,FB20,Automatic,Hybrid,Non Pano,M20A,Non GGN,Non CSA,MAGH10R-BRXLBD-C4
3,22428,MHFDB8FS2R0512248,1G0,GUN155L-SDTLXV,DT,D4,89,20240821.0,80758.0,20240821,104000,20240821,111300,GUN155L-SDTLXV-DT,D9DT,D9,GUN155L-SDTLXV,DT,FM42,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,Non CSA,GUN155L-SDTLXV-DT
4,22458,MHFJB8EM2R1141883,1G0,GUN142R-MDTSXD,JG,20,218,20240821.0,82406.0,20240821,113900,20240821,125700,GUN142R-MDTSXD-JG,D1JG,D1,GUN142R-MDTSXD,JG,FB41,Automatic,Conventional,Non Pano,GD-Diesel,Non GGN,Non CSA,GUN142R-MDTSXD-JG
5,22459,MHFABBAA4R0430441,1G0,MAGH10L-BRXLBV,DT,L1,4Q0,20240821.0,82528.0,20240821,114000,20240821,125800,MAGH10L-BRXLBV-DT,D8DT,D8,MAGH10L-BRXLBV,DT,FB20,Automatic,Hybrid,Non Pano,M20A,Non GGN,CSA,MAGH10L-BRXLBV-DT
7,22460,MHFDX8FS6R0141631,1G0,TGN156L-SDTLKV,C7,G4,89,20240821.0,82818.0,20240821,114100,20240821,125900,TGN156L-SDTLKV-C7,D9C7,D9,TGN156L-SDTLKV,C7,FM42,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,Non CSA,TGN156L-SDTLKV-C7
8,22474,MHFABBAA5R0430447,1G0,MAGH10L-BPXHBU,BT,MH,89,20240821.0,85035.0,20240821,124800,20240821,132100,MAGH10L-BPXHBU-BT,D8BT,D8,MAGH10L-BPXHBU,BT,LA40,Automatic,Hybrid,PANO,M20A,Non GGN,CSA,MAGH10L-BPXHBU-BT
9,22475,MHFJB8EM8R1141886,1G0,GUN142R-MDTSXD,JG,20,218,20240821.0,85203.0,20240821,125100,20240821,132400,GUN142R-MDTSXD-JG,D1JG,D1,GUN142R-MDTSXD,JG,FB41,Automatic,Conventional,Non Pano,GD-Diesel,Non GGN,Non CSA,GUN142R-MDTSXD-JG
10,22476,MHFAAAAA3R0034675,1G0,MAGA10R-BRXMBD,BK,0,218,20240821.0,85327.0,20240821,125100,20240821,132400,MAGA10R-BRXMBD-BK,D8BK,D8,MAGA10R-BRXMBD,BK,FA20,Automatic,Conventional,Non Pano,M20A,Non GGN,Non CSA,MAGA10R-BRXMBD-BK
12,22477,MHFJB8EMXR1141887,1G0,GUN142R-MDMSXD,HR,20,40,20240821.0,85459.0,20240821,125300,20240821,132600,GUN142R-MDMSXD-HR,D1HR,D1,GUN142R-MDMSXD,HR,FB41,Manual,Conventional,Non Pano,GD-Diesel,Non GGN,Non CSA,GUN142R-MDMSXD-HR


In [158]:
# g0_1_mapped.to_excel("../output/j0_1_mapped.xlsx")

### Mapping vehicle to C-MPV / IMV / Bypass


In [22]:
# Check for Bypass

# Calculate differences with the previous and next row
g0_1_mapped['diff_prev'] = g0_1_mapped['body_no'] - g0_1_mapped['body_no'].shift(1)
g0_1_mapped['diff_next'] = g0_1_mapped['body_no'].shift(-1) - g0_1_mapped['body_no']

# Update 'type' to 'delay' where the difference is greater than 18, else "Not Assigned"
g0_1_mapped['type'] = g0_1_mapped.apply(
    lambda row: 'delay' if (abs(pd.notna(row['diff_prev']) and row['diff_prev']) > 18) and 
                           (abs(pd.notna(row['diff_next']) and row['diff_next']) > 18) else "Not Assigned",
    axis=1
)

# Optionally, you might want to drop the auxiliary columns 'diff_prev' and 'diff_next' after updating 'type'
g0_1_mapped.drop(columns=['diff_prev', 'diff_next'], inplace=True)

g0_1_mapped.head(20)


,body_no,vin_no,curr_tp,model_no,p_sfx,s_sfx,ext_color,proc_date_g0,proc_time_g0,plan_date_j0,plan_time_j0,plan_date_k0,plan_time_k0,katashiki_sfx_alc,MdlXSFX,IDLINECD,Katashiki,SFX,Trim,Transmisi,Hybrid,Panoramic,Engine,GGN,CSA,katashiki_sfx_rest,type
0,22444,MHFBA3FS9R1055963,1G0,GUN156L-SDTHXM,BN,F7,2PS,20240821.0,75846.0,20240821,153600,20240821,211400,GUN156L-SDTHXM-BN,D9BN,D9,GUN156L-SDTHXM,BN,LJ24,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,CSA,GUN156L-SDTHXM-BN,Not Assigned
1,22447,MHFABAAA4R0026324,1G0,MAGH10R-BRXLBD,C4,0,218,20240821.0,80135.0,20240821,110800,20240821,114100,MAGH10R-BRXLBD-C4,D8C4,D8,MAGH10R-BRXLBD,C4,FB20,Automatic,Hybrid,Non Pano,M20A,Non GGN,Non CSA,MAGH10R-BRXLBD-C4,Not Assigned
3,22428,MHFDB8FS2R0512248,1G0,GUN155L-SDTLXV,DT,D4,89,20240821.0,80758.0,20240821,104000,20240821,111300,GUN155L-SDTLXV-DT,D9DT,D9,GUN155L-SDTLXV,DT,FM42,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,Non CSA,GUN155L-SDTLXV-DT,delay
4,22458,MHFJB8EM2R1141883,1G0,GUN142R-MDTSXD,JG,20,218,20240821.0,82406.0,20240821,113900,20240821,125700,GUN142R-MDTSXD-JG,D1JG,D1,GUN142R-MDTSXD,JG,FB41,Automatic,Conventional,Non Pano,GD-Diesel,Non GGN,Non CSA,GUN142R-MDTSXD-JG,Not Assigned
5,22459,MHFABBAA4R0430441,1G0,MAGH10L-BRXLBV,DT,L1,4Q0,20240821.0,82528.0,20240821,114000,20240821,125800,MAGH10L-BRXLBV-DT,D8DT,D8,MAGH10L-BRXLBV,DT,FB20,Automatic,Hybrid,Non Pano,M20A,Non GGN,CSA,MAGH10L-BRXLBV-DT,Not Assigned
7,22460,MHFDX8FS6R0141631,1G0,TGN156L-SDTLKV,C7,G4,89,20240821.0,82818.0,20240821,114100,20240821,125900,TGN156L-SDTLKV-C7,D9C7,D9,TGN156L-SDTLKV,C7,FM42,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,Non CSA,TGN156L-SDTLKV-C7,Not Assigned
8,22474,MHFABBAA5R0430447,1G0,MAGH10L-BPXHBU,BT,MH,89,20240821.0,85035.0,20240821,124800,20240821,132100,MAGH10L-BPXHBU-BT,D8BT,D8,MAGH10L-BPXHBU,BT,LA40,Automatic,Hybrid,PANO,M20A,Non GGN,CSA,MAGH10L-BPXHBU-BT,Not Assigned
9,22475,MHFJB8EM8R1141886,1G0,GUN142R-MDTSXD,JG,20,218,20240821.0,85203.0,20240821,125100,20240821,132400,GUN142R-MDTSXD-JG,D1JG,D1,GUN142R-MDTSXD,JG,FB41,Automatic,Conventional,Non Pano,GD-Diesel,Non GGN,Non CSA,GUN142R-MDTSXD-JG,Not Assigned
10,22476,MHFAAAAA3R0034675,1G0,MAGA10R-BRXMBD,BK,0,218,20240821.0,85327.0,20240821,125100,20240821,132400,MAGA10R-BRXMBD-BK,D8BK,D8,MAGA10R-BRXMBD,BK,FA20,Automatic,Conventional,Non Pano,M20A,Non GGN,Non CSA,MAGA10R-BRXMBD-BK,Not Assigned
12,22477,MHFJB8EMXR1141887,1G0,GUN142R-MDMSXD,HR,20,40,20240821.0,85459.0,20240821,125300,20240821,132600,GUN142R-MDMSXD-HR,D1HR,D1,GUN142R-MDMSXD,HR,FB41,Manual,Conventional,Non Pano,GD-Diesel,Non GGN,Non CSA,GUN142R-MDMSXD-HR,Not Assigned


In [23]:
# Check for C-MPV

# Import list of cmpv katashiki
cmpv = pd.read_excel(f"katashiki_c-mpv.xlsx")

# Create a set from cmpv for quick membership checking
cmpv_set = set(cmpv['katashiki_sfx'])

g0_1_mapped.loc[
    (g0_1_mapped['type'] == "Not Assigned") & g0_1_mapped['katashiki_sfx_alc'].isin(cmpv_set),
    'type'
] = 'cmpv'

g0_1_mapped


,body_no,vin_no,curr_tp,model_no,p_sfx,s_sfx,ext_color,proc_date_g0,proc_time_g0,plan_date_j0,plan_time_j0,plan_date_k0,plan_time_k0,katashiki_sfx_alc,MdlXSFX,IDLINECD,Katashiki,SFX,Trim,Transmisi,Hybrid,Panoramic,Engine,GGN,CSA,katashiki_sfx_rest,type
0,22444,MHFBA3FS9R1055963,1G0,GUN156L-SDTHXM,BN,F7,2PS,20240821.0,75846.0,20240821,153600,20240821,211400,GUN156L-SDTHXM-BN,D9BN,D9,GUN156L-SDTHXM,BN,LJ24,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,CSA,GUN156L-SDTHXM-BN,Not Assigned
1,22447,MHFABAAA4R0026324,1G0,MAGH10R-BRXLBD,C4,0,218,20240821.0,80135.0,20240821,110800,20240821,114100,MAGH10R-BRXLBD-C4,D8C4,D8,MAGH10R-BRXLBD,C4,FB20,Automatic,Hybrid,Non Pano,M20A,Non GGN,Non CSA,MAGH10R-BRXLBD-C4,cmpv
3,22428,MHFDB8FS2R0512248,1G0,GUN155L-SDTLXV,DT,D4,89,20240821.0,80758.0,20240821,104000,20240821,111300,GUN155L-SDTLXV-DT,D9DT,D9,GUN155L-SDTLXV,DT,FM42,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,Non CSA,GUN155L-SDTLXV-DT,delay
4,22458,MHFJB8EM2R1141883,1G0,GUN142R-MDTSXD,JG,20,218,20240821.0,82406.0,20240821,113900,20240821,125700,GUN142R-MDTSXD-JG,D1JG,D1,GUN142R-MDTSXD,JG,FB41,Automatic,Conventional,Non Pano,GD-Diesel,Non GGN,Non CSA,GUN142R-MDTSXD-JG,Not Assigned
5,22459,MHFABBAA4R0430441,1G0,MAGH10L-BRXLBV,DT,L1,4Q0,20240821.0,82528.0,20240821,114000,20240821,125800,MAGH10L-BRXLBV-DT,D8DT,D8,MAGH10L-BRXLBV,DT,FB20,Automatic,Hybrid,Non Pano,M20A,Non GGN,CSA,MAGH10L-BRXLBV-DT,cmpv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,22489,MHFAA3GS1R0751247,1G0,GUN166L-SDTHXM,CL,F5,2PS,20240821.0,110015.0,20240821,221500,20240821,224800,GUN166L-SDTHXM-CL,D9CL,D9,GUN166L-SDTHXM,CL,LF31,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,CSA,GUN166L-SDTHXM-CL,delay
127,22554,MHFKX8FS800321932,1G0,TGN156L-SDTSKN,EQ,P3,218,20240821.0,110201.0,20240821,150500,20240821,154900,TGN156L-SDTSKN-EQ,D9EQ,D9,TGN156L-SDTSKN,EQ,FN22,Automatic,Conventional,Non Pano,TR-Gasoline,Non GGN,Non CSA,TGN156L-SDTSKN-EQ,Not Assigned
128,22555,MHFABAAAXR0026344,1G0,MAGH10R-BRXLBD,C4,1,89,20240821.0,110342.0,20240821,150600,20240821,155100,MAGH10R-BRXLBD-C4,D8C4,D8,MAGH10R-BRXLBD,C4,FB20,Automatic,Hybrid,Non Pano,M20A,Non GGN,Non CSA,MAGH10R-BRXLBD-C4,cmpv
130,22556,MHFBU3FS4S0352903,1G0,GGN155L-SDTHK,AA,P7,218,20240821.0,110505.0,20240821,150700,20240821,155200,GGN155L-SDTHK-AA,D9AA,D9,GGN155L-SDTHK,AA,FN22,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,CSA,GGN155L-SDTHK-AA,Not Assigned


In [24]:
# Check for IMV

# Import list of cmpv katashiki
imv = pd.read_excel(f"katashiki_imv.xlsx")

# Create a set from cmpv for quick membership checking
imv_set = set(imv['katashiki_sfx'])

g0_1_mapped.loc[
    (g0_1_mapped['type'] == "Not Assigned") & g0_1_mapped['katashiki_sfx_alc'].isin(imv_set),
    'type'
] = 'imv'

g0_1_mapped


,body_no,vin_no,curr_tp,model_no,p_sfx,s_sfx,ext_color,proc_date_g0,proc_time_g0,plan_date_j0,plan_time_j0,plan_date_k0,plan_time_k0,katashiki_sfx_alc,MdlXSFX,IDLINECD,Katashiki,SFX,Trim,Transmisi,Hybrid,Panoramic,Engine,GGN,CSA,katashiki_sfx_rest,type
0,22444,MHFBA3FS9R1055963,1G0,GUN156L-SDTHXM,BN,F7,2PS,20240821.0,75846.0,20240821,153600,20240821,211400,GUN156L-SDTHXM-BN,D9BN,D9,GUN156L-SDTHXM,BN,LJ24,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,CSA,GUN156L-SDTHXM-BN,imv
1,22447,MHFABAAA4R0026324,1G0,MAGH10R-BRXLBD,C4,0,218,20240821.0,80135.0,20240821,110800,20240821,114100,MAGH10R-BRXLBD-C4,D8C4,D8,MAGH10R-BRXLBD,C4,FB20,Automatic,Hybrid,Non Pano,M20A,Non GGN,Non CSA,MAGH10R-BRXLBD-C4,cmpv
3,22428,MHFDB8FS2R0512248,1G0,GUN155L-SDTLXV,DT,D4,89,20240821.0,80758.0,20240821,104000,20240821,111300,GUN155L-SDTLXV-DT,D9DT,D9,GUN155L-SDTLXV,DT,FM42,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,Non CSA,GUN155L-SDTLXV-DT,delay
4,22458,MHFJB8EM2R1141883,1G0,GUN142R-MDTSXD,JG,20,218,20240821.0,82406.0,20240821,113900,20240821,125700,GUN142R-MDTSXD-JG,D1JG,D1,GUN142R-MDTSXD,JG,FB41,Automatic,Conventional,Non Pano,GD-Diesel,Non GGN,Non CSA,GUN142R-MDTSXD-JG,imv
5,22459,MHFABBAA4R0430441,1G0,MAGH10L-BRXLBV,DT,L1,4Q0,20240821.0,82528.0,20240821,114000,20240821,125800,MAGH10L-BRXLBV-DT,D8DT,D8,MAGH10L-BRXLBV,DT,FB20,Automatic,Hybrid,Non Pano,M20A,Non GGN,CSA,MAGH10L-BRXLBV-DT,cmpv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,22489,MHFAA3GS1R0751247,1G0,GUN166L-SDTHXM,CL,F5,2PS,20240821.0,110015.0,20240821,221500,20240821,224800,GUN166L-SDTHXM-CL,D9CL,D9,GUN166L-SDTHXM,CL,LF31,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,CSA,GUN166L-SDTHXM-CL,delay
127,22554,MHFKX8FS800321932,1G0,TGN156L-SDTSKN,EQ,P3,218,20240821.0,110201.0,20240821,150500,20240821,154900,TGN156L-SDTSKN-EQ,D9EQ,D9,TGN156L-SDTSKN,EQ,FN22,Automatic,Conventional,Non Pano,TR-Gasoline,Non GGN,Non CSA,TGN156L-SDTSKN-EQ,imv
128,22555,MHFABAAAXR0026344,1G0,MAGH10R-BRXLBD,C4,1,89,20240821.0,110342.0,20240821,150600,20240821,155100,MAGH10R-BRXLBD-C4,D8C4,D8,MAGH10R-BRXLBD,C4,FB20,Automatic,Hybrid,Non Pano,M20A,Non GGN,Non CSA,MAGH10R-BRXLBD-C4,cmpv
130,22556,MHFBU3FS4S0352903,1G0,GGN155L-SDTHK,AA,P7,218,20240821.0,110505.0,20240821,150700,20240821,155200,GGN155L-SDTHK-AA,D9AA,D9,GGN155L-SDTHK,AA,FN22,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,CSA,GGN155L-SDTHK-AA,Not Assigned


In [25]:
# Check if all rows are assigned
g0_1_mapped.value_counts('type')

type
cmpv            42
imv             35
delay            9
Not Assigned     1
Name: count, dtype: int64

## Sequence Optimization & Lane Assignments

- Goal: Assign every vehicle in J0 to their respective lane based on production rules & restrictions


### Rules & Restrictions


1. Lanes:

- Lane 1 & 2: C-MPV => Can be identified from Katashiki
- Lane 3 & 4: IMV => Fortuner & Innova
- Lane 5: Bypass => Delay

2. HV

- Two hybrid cars cannot be processed right after each other, must have 1 car non-hybrid processed in the middle
- j0_1_mapped['Hybrid'] == "Hybrid"

3. Panoramic

- There must be 3 non-panoramic cars in between each panoramic cars
- j0_1_mapped['Panoramic'] == "PANO"

4. CSA

- There must be 4 Non-CSA cars in between each CSA cars
- j0_1_mapped['CSA'] == "CSA"

- Dummy delay rule: the proc_time >= 2 hours after the initial plan_time
- Logic: proc_time_j0 - plan_time_j0 > 20000


### Data Preparation


In [163]:
# # Take only the first 30
# g0_mapped_30 = g0_1_mapped[:30]

In [238]:
# Take only the first 20
# g0_mapped_20 = g0_1_mapped[:20]
print(len(g0_1_mapped))

87


In [336]:
# add delay status column
g0_1_mapped['delay_status'] = g0_1_mapped['type'].apply(lambda x: 'delay' if x == 'delay' else '')
g0_1_mapped['type'] = g0_1_mapped['type'].replace('delay', '')

# add model column
g0_1_mapped['model'] = g0_1_mapped['IDLINECD'].apply(lambda x: 'Fortuner' if x == 'D9' else ('Zennix' if x == 'D8' else 'Innova'))

# rearrange column order
g0_1_mapped = g0_1_mapped[['body_no', 'vin_no', 'curr_tp', 'model_no', 'p_sfx', 's_sfx',
       'ext_color', 'proc_date_g0', 'proc_time_g0', 'plan_date_j0',
       'plan_time_j0', 'plan_date_k0', 'plan_time_k0', 'katashiki_sfx_alc',
       'MdlXSFX', 'IDLINECD', 'Katashiki', 'SFX', 'Trim', 'Transmisi',
       'Hybrid', 'Panoramic', 'Engine', 'GGN', 'CSA', 'katashiki_sfx_rest',
       'type', 'model','delay_status', 'og_seq']]

g0_1_mapped

['D9' 'D8' 'D1']


In [316]:
# Take only the first 15
g0_mapped_15 = g0_1_mapped[:30]

In [330]:
# Select distinct values from a column (e.g., 'type')
# distinct_values = df['type'].unique()

# Display the distinct values
print(g0_mapped_15['IDLINECD'].unique())

['D9' 'D8' 'D1']


In [ ]:
g0_mapped_15['delay_status'] = g0_mapped_15['type'].apply(lambda x: 'delay' if x == 'delay' else '')

# Remove 'delay' from the 'type' column (can replace with None or empty string)
g0_mapped_15['type'] = g0_mapped_15['type'].replace('delay', '')

g0_mapped_15['model'] = g0_mapped_15['IDLINECD'].apply(lambda x: 'Fortuner' if x == 'D9' else ('Zennix' if x == 'D8' else 'Innova'))


In [ ]:
g0_mapped_15

In [333]:
# List all column names from a DataFrame
column_names = g0_mapped_15.columns

# Display the list of column names
print(column_names)

Index(['body_no', 'vin_no', 'curr_tp', 'model_no', 'p_sfx', 's_sfx',
       'ext_color', 'proc_date_g0', 'proc_time_g0', 'plan_date_j0',
       'plan_time_j0', 'plan_date_k0', 'plan_time_k0', 'katashiki_sfx_alc',
       'MdlXSFX', 'IDLINECD', 'Katashiki', 'SFX', 'Trim', 'Transmisi',
       'Hybrid', 'Panoramic', 'Engine', 'GGN', 'CSA', 'katashiki_sfx_rest',
       'type', 'og_seq', 'delay_status', 'model'],
      dtype='object')


In [ ]:
g0_mapped_15 = g0_mapped_15[['body_no', 'vin_no', 'curr_tp', 'model_no', 'p_sfx', 's_sfx',
       'ext_color', 'proc_date_g0', 'proc_time_g0', 'plan_date_j0',
       'plan_time_j0', 'plan_date_k0', 'plan_time_k0', 'katashiki_sfx_alc',
       'MdlXSFX', 'IDLINECD', 'Katashiki', 'SFX', 'Trim', 'Transmisi',
       'Hybrid', 'Panoramic', 'Engine', 'GGN', 'CSA', 'katashiki_sfx_rest',
       'type', 'model','delay_status', 'og_seq']]

g0_mapped_15

### Constraint Violation Checker with CP

- Currently, if we sort the rows by `proc_time_g0`, the table's already sorted by `body_no` as well. It means that the sequence / order is already correct or close to the ones in A0 where's no delay. This also means that all constraints are already met. Right now, we just have to check and make sure if the constraints are still met

- If all the constraints are already met, berarti langsung bisa di numbering 1 2 3 4

- Future: Ask if kalo gak 1 2 khusus untuk cmpv, dll gapapa gak


#### Constraints Prioritization (Please Confirm)

1. Delay
2. HV
3. Panoramic
4. CSA


In [167]:
# # Constraint Checker Function v02 - with violation location added

# def check_constraints(df):
#     # Initialize the result and failure location columns for all rows
#     df['constraint_checker'] = "Pass"
#     df['failed_const_loc'] = ""

#     # Helper function to add failure types without duplication
#     def add_failure(row_index, failure_type):
#         if failure_type not in df.at[row_index, 'failed_const_loc']:
#             if df.at[row_index, 'failed_const_loc']:
#                 df.at[row_index, 'failed_const_loc'] += ", "
#             df.at[row_index, 'failed_const_loc'] += failure_type

#     # Check Hybrid constraint
#     hybrid_indices = df.index[df['Hybrid'] == 'Hybrid'].tolist()
#     for i, idx in enumerate(hybrid_indices):
#         if i > 0:  # Check if there's a previous hybrid
#             if (idx - hybrid_indices[i - 1]) <= 2:  # Should be more than 1 non-hybrid in between
#                 df.at[idx, 'constraint_checker'] = "Failed"
#                 df.at[hybrid_indices[i - 1], 'constraint_checker'] = "Failed"
#                 add_failure(idx, 'Hybrid')
#                 add_failure(hybrid_indices[i - 1], 'Hybrid')

#     # Check Panoramic constraint
#     panoramic_indices = df.index[df['Panoramic'] == 'PANO'].tolist()
#     for i, idx in enumerate(panoramic_indices):
#         if i > 0:  # Check if there's a previous panoramic
#             if (idx - panoramic_indices[i - 1]) <= 4:  # Should be more than 3 non-panoramic in between
#                 df.at[idx, 'constraint_checker'] = "Failed"
#                 df.at[panoramic_indices[i - 1], 'constraint_checker'] = "Failed"
#                 add_failure(idx, 'Panoramic')
#                 add_failure(panoramic_indices[i - 1], 'Panoramic')

#     # Check CSA constraint
#     csa_indices = df.index[df['CSA'] == 'CSA'].tolist()
#     for i, idx in enumerate(csa_indices):
#         if i > 0:  # Check if there's a previous CSA
#             if (idx - csa_indices[i - 1]) <= 5:  # Should be more than 4 non-CSA in between
#                 df.at[idx, 'constraint_checker'] = "Failed"
#                 df.at[csa_indices[i - 1], 'constraint_checker'] = "Failed"
#                 add_failure(idx, 'CSA')
#                 add_failure(csa_indices[i - 1], 'CSA')

#     # Clean up 'failed_const_loc' by removing trailing commas and spaces
#     df['failed_const_loc'] = df['failed_const_loc'].apply(lambda x: x.strip(", "))

In [122]:
def check_constraints(df, hybrid_col='Hybrid', pano_col='Panoramic', csa_col='CSA'):
    df['constraint_checker'] = ''
    df['failed_const_loc'] = ''

    # Function to check and append the failure reason
    def check_failure(df, column_name, row, j_range, reason):
        for j in j_range:
            if (row - j >= 0 and df.loc[row - j, column_name] == df.loc[row, column_name]) or (row + j < len(df) and df.loc[row + j, column_name] == df.loc[row, column_name]):
                if df.loc[row, 'failed_const_loc'] == '':
                    df.loc[row, 'failed_const_loc'] = reason
                else:
                    df.loc[row, 'failed_const_loc'] += f", {reason}"

    # Loop to check and append failure reasons for 'hybrid', 'pano', and 'csa'
    for i in range(len(df)):
        if df.loc[i, hybrid_col] == 'Hybrid':
            check_failure(df, hybrid_col, i, range(1, 2), 'Hybrid')

        if df.loc[i, pano_col] == 'PANO':
            check_failure(df, pano_col, i, range(1, 4), 'Panoramic')

        if df.loc[i, csa_col] == 'CSA':
            check_failure(df, csa_col, i, range(1, 5), 'CSA')

    # # Fill remaining rows without failure reason with 'ok'
    # df['failed_const_loc'] = df['failed_const_loc'].replace('', 'Pass')
    
    # Recalculating 'status' based on the failure reason
    df['constraint_checker'] = df['failed_const_loc'].apply(lambda x: 'Failed' if x != '' else 'Pass')

In [318]:
# Apply the function
g0_constraint_checked = g0_mapped_15.copy()

g0_constraint_checked = g0_constraint_checked.reset_index(drop=True)

check_constraints(g0_constraint_checked)

g0_constraint_checked.value_counts('constraint_checker')

constraint_checker
Pass      17
Failed    13
Name: count, dtype: int64

In [319]:
g0_constraint_checked

,body_no,vin_no,curr_tp,model_no,p_sfx,s_sfx,ext_color,proc_date_g0,proc_time_g0,plan_date_j0,plan_time_j0,plan_date_k0,plan_time_k0,katashiki_sfx_alc,MdlXSFX,IDLINECD,Katashiki,SFX,Trim,Transmisi,Hybrid,Panoramic,Engine,GGN,CSA,katashiki_sfx_rest,type,constraint_checker,failed_const_loc
0,22444,MHFBA3FS9R1055963,1G0,GUN156L-SDTHXM,BN,F7,2PS,20240821.0,75846.0,20240821,153600,20240821,211400,GUN156L-SDTHXM-BN,D9BN,D9,GUN156L-SDTHXM,BN,LJ24,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,CSA,GUN156L-SDTHXM-BN,imv,Failed,CSA
1,22447,MHFABAAA4R0026324,1G0,MAGH10R-BRXLBD,C4,0,218,20240821.0,80135.0,20240821,110800,20240821,114100,MAGH10R-BRXLBD-C4,D8C4,D8,MAGH10R-BRXLBD,C4,FB20,Automatic,Hybrid,Non Pano,M20A,Non GGN,Non CSA,MAGH10R-BRXLBD-C4,cmpv,Pass,
2,22428,MHFDB8FS2R0512248,1G0,GUN155L-SDTLXV,DT,D4,89,20240821.0,80758.0,20240821,104000,20240821,111300,GUN155L-SDTLXV-DT,D9DT,D9,GUN155L-SDTLXV,DT,FM42,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,Non CSA,GUN155L-SDTLXV-DT,delay,Pass,
3,22458,MHFJB8EM2R1141883,1G0,GUN142R-MDTSXD,JG,20,218,20240821.0,82406.0,20240821,113900,20240821,125700,GUN142R-MDTSXD-JG,D1JG,D1,GUN142R-MDTSXD,JG,FB41,Automatic,Conventional,Non Pano,GD-Diesel,Non GGN,Non CSA,GUN142R-MDTSXD-JG,imv,Pass,
4,22459,MHFABBAA4R0430441,1G0,MAGH10L-BRXLBV,DT,L1,4Q0,20240821.0,82528.0,20240821,114000,20240821,125800,MAGH10L-BRXLBV-DT,D8DT,D8,MAGH10L-BRXLBV,DT,FB20,Automatic,Hybrid,Non Pano,M20A,Non GGN,CSA,MAGH10L-BRXLBV-DT,cmpv,Failed,"CSA, CSA"
5,22460,MHFDX8FS6R0141631,1G0,TGN156L-SDTLKV,C7,G4,89,20240821.0,82818.0,20240821,114100,20240821,125900,TGN156L-SDTLKV-C7,D9C7,D9,TGN156L-SDTLKV,C7,FM42,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,Non CSA,TGN156L-SDTLKV-C7,imv,Pass,
6,22474,MHFABBAA5R0430447,1G0,MAGH10L-BPXHBU,BT,MH,89,20240821.0,85035.0,20240821,124800,20240821,132100,MAGH10L-BPXHBU-BT,D8BT,D8,MAGH10L-BPXHBU,BT,LA40,Automatic,Hybrid,PANO,M20A,Non GGN,CSA,MAGH10L-BPXHBU-BT,cmpv,Failed,"CSA, CSA"
7,22475,MHFJB8EM8R1141886,1G0,GUN142R-MDTSXD,JG,20,218,20240821.0,85203.0,20240821,125100,20240821,132400,GUN142R-MDTSXD-JG,D1JG,D1,GUN142R-MDTSXD,JG,FB41,Automatic,Conventional,Non Pano,GD-Diesel,Non GGN,Non CSA,GUN142R-MDTSXD-JG,imv,Pass,
8,22476,MHFAAAAA3R0034675,1G0,MAGA10R-BRXMBD,BK,0,218,20240821.0,85327.0,20240821,125100,20240821,132400,MAGA10R-BRXMBD-BK,D8BK,D8,MAGA10R-BRXMBD,BK,FA20,Automatic,Conventional,Non Pano,M20A,Non GGN,Non CSA,MAGA10R-BRXMBD-BK,cmpv,Pass,
9,22477,MHFJB8EMXR1141887,1G0,GUN142R-MDMSXD,HR,20,40,20240821.0,85459.0,20240821,125300,20240821,132600,GUN142R-MDMSXD-HR,D1HR,D1,GUN142R-MDMSXD,HR,FB41,Manual,Conventional,Non Pano,GD-Diesel,Non GGN,Non CSA,GUN142R-MDMSXD-HR,imv,Pass,


### Sequence Optimization with GA


In [320]:
# Reset index
g0_mapped_15.reset_index(drop=True, inplace=True)

In [321]:
# Initialize the original sequence
g0_mapped_15['og_seq'] = g0_mapped_15.index

C:\Users\msubki\AppData\Local\Temp\ipykernel_23160\4070664635.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g0_mapped_15['og_seq'] = g0_mapped_15.index


In [322]:
def evaluate_fitness(sequence, df):
    try:
        df_temp = df.iloc[sequence].reset_index(drop=True)
        check_constraints(df_temp)  # Ensure constraints are checked
        movement_penalty = sum(abs(seq - idx) > 4 for idx, seq in enumerate(sequence))
        fitness = -df_temp['constraint_checker'].tolist().count("Failed")- movement_penalty

        # Prioritize delayed vehicles
        for idx, (seq, row) in enumerate(zip(sequence, df_temp.itertuples())):
            original_idx = df.index[df['vin_no'] == row.vin_no].tolist()[0]
            if row.type == 'delay':
                if seq < original_idx:  # Reward for moving up
                    fitness += 5
                elif seq > original_idx:  # Penalty for moving down
                    fitness -= 10
            
            # Reward for sorting by body_no
            # if idx > 0 and row.body_no < df_temp.iloc[idx - 1].body_no:
            #     fitness += 2

        return fitness, df_temp
    except IndexError as e:
        print(f"Error with sequence: {sequence}")
        raise e

In [277]:
def crossover(parent1, parent2):
    length = len(parent1)
    child = [None] * length  # Initialize child with None to ensure all positions are filled uniquely
    used_indices = set()
    
    for idx in range(length):
        options = []
        
        # Collect valid options from both parents
        for p in [parent1, parent2]:
            if p[idx] not in used_indices and (idx - 4 <= p[idx] <= idx + 4):
                options.append(p[idx])
        
        # Select a random valid option if available
        if options:
            choice = random.choice(options)
        else:
            # Fallback: find any unused index that fits the movement constraint, if possible
            possible_choices = [i for i in range(max(0, idx - 4), min(length, idx + 4 + 1)) if i not in used_indices]
            if possible_choices:
                choice = random.choice(possible_choices)
            else:
                # If no possible choices are found, pick any unused index
                unused = list(set(range(length)) - used_indices)
                if unused:
                    choice = random.choice(unused)
                else:
                    raise ValueError("Should never run out of unused indices if all conditions are correct.")
        
        child[idx] = choice
        used_indices.add(choice)
    
    return child

def mutate(sequence):
    idx1 = random.randint(0, len(sequence) - 1)
    # Limit mutation choices to within 4 steps of idx1 and ensure no duplication
    choices = [idx for idx in range(max(0, idx1 - 4), min(len(sequence), idx1 + 5)) if idx != idx1]
    if choices:
        idx2 = random.choice(choices)
        sequence[idx1], sequence[idx2] = sequence[idx2], sequence[idx1]

In [262]:
def genetic_algorithm(df, population_size=100, generations=50, mutation_rate=0.05, max_failures=10):
    population = [list(df.index) for _ in range(population_size)]
    for _ in range(population_size):
        random.shuffle(population[_])

    best_solution = None
    best_fitness = float('-inf')
    iteration = 0
    max_iterations = 10  # Set a limit to avoid infinite loops

    while iteration < max_iterations:
        for generation in range(generations):
            new_population = []
            fitness_scores = []

            for individual in population:
                fitness, temp_df = evaluate_fitness(individual, df)
                fitness_scores.append((individual, fitness, temp_df))

            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            current_best = fitness_scores[0]
            if current_best[1] > best_fitness:
                best_fitness = current_best[1]
                best_solution = current_best[0]
                best_df = current_best[2]

            survivors = fitness_scores[:population_size // 2]
            new_population = [x[0] for x in survivors]

            while len(new_population) < population_size:
                parent1, parent2 = random.sample(new_population, 2)
                child = crossover(parent1, parent2)
                if random.random() < mutation_rate:
                    mutate(child)
                new_population.append(child)

            population = new_population

        # Check if the solution meets all criteria
        best_df['new_seq'] = best_solution
        best_df['seq_diff'] = best_df['og_seq'] - best_df['new_seq']
        check_constraints(best_df)
        failed_count = best_df['constraint_checker'].tolist().count("Failed")

        # If criteria are met, break the loop
        if failed_count <= max_failures and all(abs(x) <= 4 for x in best_df['seq_diff']):
            break

        iteration += 1  # Increment the iteration if not yet satisfactory

    return best_df

In [323]:
# Run the genetic algorithm
optimized_df = genetic_algorithm(g0_mapped_15)

In [296]:
# Add final sequence and check constraints
optimized_df['new_seq'] = optimized_df.index
check_constraints(optimized_df)  # Update constraints check on the final sequence

In [297]:
optimized_df.value_counts('constraint_checker')

constraint_checker
Pass      6
Failed    4
Name: count, dtype: int64

In [327]:
# Adding the seq_diff column
optimized_df['seq_diff'] = optimized_df['og_seq'] - optimized_df['new_seq']
optimized_df['seq_diff'] = optimized_df['seq_diff'].apply(lambda x: f"+{x}" if x > 0 else str(x))
# optimized_df['seq_status'] = np.where(optimized_df['seq_diff'].astype(int) < 0, 'delay', 'ok')

In [328]:
# Output the required columns

# optimized_df[['body_no', 'vin_no', 'og_seq', 'new_seq', 'constraint_checker', 'failed_const_loc']]

data = optimized_df[['body_no', 'vin_no', 'og_seq', 'new_seq', 'seq_diff','type', 'Hybrid', 'Panoramic', 'CSA', 'constraint_checker', 'failed_const_loc']]
data['delay_status'] = data['type'].apply(lambda x: 'delay' if x == 'delay' else '')

# Remove 'delay' from the 'type' column (can replace with None or empty string)
data['type'] = data['type'].replace('delay', '')

data

C:\Users\msubki\AppData\Local\Temp\ipykernel_23160\355235286.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['delay_status'] = data['type'].apply(lambda x: 'delay' if x == 'delay' else '')
C:\Users\msubki\AppData\Local\Temp\ipykernel_23160\355235286.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['type'] = data['type'].replace('delay', '')


,body_no,vin_no,og_seq,new_seq,seq_diff,type,Hybrid,Panoramic,CSA,constraint_checker,failed_const_loc,delay_status
0,22447,MHFABAAA4R0026324,1,1,0,cmpv,Hybrid,Non Pano,Non CSA,Pass,,
1,22460,MHFDX8FS6R0141631,5,5,0,imv,Conventional,Non Pano,Non CSA,Pass,,
2,22474,MHFABBAA5R0430447,6,6,0,cmpv,Hybrid,PANO,CSA,Pass,,
3,22475,MHFJB8EM8R1141886,7,7,0,imv,Conventional,Non Pano,Non CSA,Pass,,
4,22476,MHFAAAAA3R0034675,8,8,0,cmpv,Conventional,Non Pano,Non CSA,Pass,,
5,22428,MHFDB8FS2R0512248,2,2,0,,Conventional,Non Pano,Non CSA,Pass,,delay
6,22477,MHFJB8EMXR1141887,9,9,0,imv,Conventional,Non Pano,Non CSA,Pass,,
7,22458,MHFJB8EM2R1141883,3,3,0,imv,Conventional,Non Pano,Non CSA,Pass,,
8,22480,MHFABAAA3R0026329,11,11,0,cmpv,Hybrid,PANO,Non CSA,Pass,,
9,22481,MHFDX3FS9S0055321,12,12,0,imv,Conventional,Non Pano,CSA,Pass,,


In [245]:
print(optimized_df['seq_status'].value_counts())

seq_status
ok       7
delay    3
Name: count, dtype: int64


0-10


In [312]:
g0_0_10_og = g0_mapped_15.copy()
g0_0_10_new = optimized_df.copy()

10-20


In [310]:
g0_10_20_og = g0_constraint_checked.copy()
g0_10_20_new = optimized_df.copy()

### reordering delay


In [268]:
df = pd.DataFrame(data)

# Find the row with 'delay' type
delay_row_idx = df[df['type'] == 'delay'].index[0]
delay_row = df.loc[delay_row_idx]

# Find the row with matching Hybrid, Panoramic, and CSA values
matching_row_idx = df[(df['Hybrid'] == delay_row['Hybrid']) &
                      (df['Panoramic'] == delay_row['Panoramic']) &
                      (df['CSA'] == delay_row['CSA']) &
                      (df['type'] != 'delay')].index

if not matching_row_idx.empty:
    matching_row_idx = matching_row_idx[0]  # Take the first matching row
    # Swap the rows
    df.iloc[[delay_row_idx, matching_row_idx]] = df.iloc[[matching_row_idx, delay_row_idx]].values

# df = df.reset_index(drop=True)
# df['new_seq'] = df.index

df

,body_no,vin_no,og_seq,new_seq,seq_diff,type,Hybrid,Panoramic,CSA,constraint_checker,failed_const_loc
0,22459,MHFABBAA4R0430441,4,0,+4,cmpv,Hybrid,Non Pano,CSA,Failed,CSA
1,22444,MHFBA3FS9R1055963,0,1,-1,imv,Conventional,Non Pano,CSA,Failed,CSA
2,22447,MHFABAAA4R0026324,1,2,-1,cmpv,Hybrid,Non Pano,Non CSA,Pass,
3,22428,MHFDB8FS2R0512248,2,6,-4,delay,Conventional,Non Pano,Non CSA,Pass,
4,22458,MHFJB8EM2R1141883,3,4,-1,imv,Conventional,Non Pano,Non CSA,Pass,
5,22476,MHFAAAAA3R0034675,8,5,+3,cmpv,Conventional,Non Pano,Non CSA,Pass,
6,22460,MHFDX8FS6R0141631,5,3,+2,imv,Conventional,Non Pano,Non CSA,Pass,
7,22474,MHFABBAA5R0430447,6,7,-1,cmpv,Hybrid,PANO,CSA,Pass,
8,22475,MHFJB8EM8R1141886,7,8,-1,imv,Conventional,Non Pano,Non CSA,Pass,
9,22477,MHFJB8EMXR1141887,9,9,0,imv,Conventional,Non Pano,Non CSA,Pass,


In [269]:
df = df.reset_index(drop=True)
df['new_seq'] = df.index
df['seq_diff'] = df['og_seq'] - df['new_seq']
df['seq_diff'] = df['seq_diff'].apply(lambda x: f"+{x}" if x > 0 else str(x))

df

,body_no,vin_no,og_seq,new_seq,seq_diff,type,Hybrid,Panoramic,CSA,constraint_checker,failed_const_loc
0,22459,MHFABBAA4R0430441,4,0,+4,cmpv,Hybrid,Non Pano,CSA,Failed,CSA
1,22444,MHFBA3FS9R1055963,0,1,-1,imv,Conventional,Non Pano,CSA,Failed,CSA
2,22447,MHFABAAA4R0026324,1,2,-1,cmpv,Hybrid,Non Pano,Non CSA,Pass,
3,22428,MHFDB8FS2R0512248,2,3,-1,delay,Conventional,Non Pano,Non CSA,Pass,
4,22458,MHFJB8EM2R1141883,3,4,-1,imv,Conventional,Non Pano,Non CSA,Pass,
5,22476,MHFAAAAA3R0034675,8,5,+3,cmpv,Conventional,Non Pano,Non CSA,Pass,
6,22460,MHFDX8FS6R0141631,5,6,-1,imv,Conventional,Non Pano,Non CSA,Pass,
7,22474,MHFABBAA5R0430447,6,7,-1,cmpv,Hybrid,PANO,CSA,Pass,
8,22475,MHFJB8EM8R1141886,7,8,-1,imv,Conventional,Non Pano,Non CSA,Pass,
9,22477,MHFJB8EMXR1141887,9,9,0,imv,Conventional,Non Pano,Non CSA,Pass,


In [180]:
optimized_df

,body_no,vin_no,curr_tp,model_no,p_sfx,s_sfx,ext_color,proc_date_g0,proc_time_g0,plan_date_j0,plan_time_j0,plan_date_k0,plan_time_k0,katashiki_sfx_alc,MdlXSFX,IDLINECD,Katashiki,SFX,Trim,Transmisi,Hybrid,Panoramic,Engine,GGN,CSA,katashiki_sfx_rest,type,og_seq,constraint_checker,failed_const_loc,new_seq,seq_diff
0,22459,MHFABBAA4R0430441,1G0,MAGH10L-BRXLBV,DT,L1,4Q0,20240821.0,82528.0,20240821,114000,20240821,125800,MAGH10L-BRXLBV-DT,D8DT,D8,MAGH10L-BRXLBV,DT,FB20,Automatic,Hybrid,Non Pano,M20A,Non GGN,CSA,MAGH10L-BRXLBV-DT,cmpv,4,Failed,CSA,0,+4
1,22458,MHFJB8EM2R1141883,1G0,GUN142R-MDTSXD,JG,20,218,20240821.0,82406.0,20240821,113900,20240821,125700,GUN142R-MDTSXD-JG,D1JG,D1,GUN142R-MDTSXD,JG,FB41,Automatic,Conventional,Non Pano,GD-Diesel,Non GGN,Non CSA,GUN142R-MDTSXD-JG,imv,3,Pass,,1,+2
2,22444,MHFBA3FS9R1055963,1G0,GUN156L-SDTHXM,BN,F7,2PS,20240821.0,75846.0,20240821,153600,20240821,211400,GUN156L-SDTHXM-BN,D9BN,D9,GUN156L-SDTHXM,BN,LJ24,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,CSA,GUN156L-SDTHXM-BN,imv,0,Failed,CSA,2,-2
3,22447,MHFABAAA4R0026324,1G0,MAGH10R-BRXLBD,C4,0,218,20240821.0,80135.0,20240821,110800,20240821,114100,MAGH10R-BRXLBD-C4,D8C4,D8,MAGH10R-BRXLBD,C4,FB20,Automatic,Hybrid,Non Pano,M20A,Non GGN,Non CSA,MAGH10R-BRXLBD-C4,cmpv,1,Pass,,3,-2
4,22475,MHFJB8EM8R1141886,1G0,GUN142R-MDTSXD,JG,20,218,20240821.0,85203.0,20240821,125100,20240821,132400,GUN142R-MDTSXD-JG,D1JG,D1,GUN142R-MDTSXD,JG,FB41,Automatic,Conventional,Non Pano,GD-Diesel,Non GGN,Non CSA,GUN142R-MDTSXD-JG,imv,7,Pass,,4,+3
5,22428,MHFDB8FS2R0512248,1G0,GUN155L-SDTLXV,DT,D4,89,20240821.0,80758.0,20240821,104000,20240821,111300,GUN155L-SDTLXV-DT,D9DT,D9,GUN155L-SDTLXV,DT,FM42,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,Non CSA,GUN155L-SDTLXV-DT,delay,2,Pass,,5,-3
6,22460,MHFDX8FS6R0141631,1G0,TGN156L-SDTLKV,C7,G4,89,20240821.0,82818.0,20240821,114100,20240821,125900,TGN156L-SDTLKV-C7,D9C7,D9,TGN156L-SDTLKV,C7,FM42,Automatic,Conventional,Non Pano,GR-Gasoline,GGN,Non CSA,TGN156L-SDTLKV-C7,imv,5,Pass,,6,-1
7,22476,MHFAAAAA3R0034675,1G0,MAGA10R-BRXMBD,BK,0,218,20240821.0,85327.0,20240821,125100,20240821,132400,MAGA10R-BRXMBD-BK,D8BK,D8,MAGA10R-BRXMBD,BK,FA20,Automatic,Conventional,Non Pano,M20A,Non GGN,Non CSA,MAGA10R-BRXMBD-BK,cmpv,8,Pass,,7,+1
8,22474,MHFABBAA5R0430447,1G0,MAGH10L-BPXHBU,BT,MH,89,20240821.0,85035.0,20240821,124800,20240821,132100,MAGH10L-BPXHBU-BT,D8BT,D8,MAGH10L-BPXHBU,BT,LA40,Automatic,Hybrid,PANO,M20A,Non GGN,CSA,MAGH10L-BPXHBU-BT,cmpv,6,Pass,,8,-2
9,22477,MHFJB8EMXR1141887,1G0,GUN142R-MDMSXD,HR,20,40,20240821.0,85459.0,20240821,125300,20240821,132600,GUN142R-MDMSXD-HR,D1HR,D1,GUN142R-MDMSXD,HR,FB41,Manual,Conventional,Non Pano,GD-Diesel,Non GGN,Non CSA,GUN142R-MDMSXD-HR,imv,9,Pass,,9,0
